In [1]:
# !pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [1]:
import os
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest, OrderBy
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Set the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'GA4_API.json'

# Initialize the GA4 client
client = BetaAnalyticsDataClient()

# Set your GA4 property ID
property_id = "361964108"

# Set up the query to fetch the required data
request = RunReportRequest(
    property='properties/' + property_id,
    dimensions=[Dimension(name="year"), Dimension(name="month")],
    metrics=[
        Metric(name="totalUsers"),
        Metric(name="activeUsers"),
        Metric(name="newUsers"),
        Metric(name="screenPageViews"),
        Metric(name="eventCount"),
        Metric(name="sessions")
    ],
    order_bys=[OrderBy(dimension={'dimension_name': 'month'})],
    date_ranges=[DateRange(start_date="2020-01-01", end_date="today")]
)

# Send the request and get the response
response = client.run_report(request)

# Format the response into a DataFrame and sort it by year and month in descending order
def format_report(response):
    row_headers = [row.dimension_values for row in response.rows]
    metric_values = [row.metric_values for row in response.rows]

    data = []
    
    for i in range(len(row_headers)):
        year = int(row_headers[i][0].value)
        month = int(row_headers[i][1].value)
        total_users = float(metric_values[i][0].value)
        active_users = float(metric_values[i][1].value)
        new_users = float(metric_values[i][2].value)
        returning_users = total_users - new_users
        pageviews = float(metric_values[i][3].value)
        hits_events = float(metric_values[i][4].value)
        sessions = float(metric_values[i][5].value)

        data.append([year, month, total_users, active_users, returning_users, new_users, pageviews, hits_events, sessions])

    df = pd.DataFrame(data, columns=[
        "Year", "Month", "Total Users", "Users/Active Users", "Returning Users", "New Users", "Pageviews", "Hits/Events", "Sessions"
    ])

    df = df.sort_values(by=["Year", "Month"], ascending=[False, False])
    df['Month-Year'] = df['Month'].apply(lambda x: f'{x:02}') + ', ' + df['Year'].astype(str)
    df = df[['Month-Year', 'Total Users', 'Users/Active Users', 'Returning Users', 'New Users', 'Pageviews', 'Hits/Events', 'Sessions']]

    return df

# Apply the formatting and sorting
df = format_report(response)

# Google Sheets API setup
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'google_sheets_api.json'

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('sheets', 'v4', credentials=creds)

# Specify your Google Sheet ID
SPREADSHEET_ID = '1faXFb6yEYzHssBFU-LH5b4YIRhBxttfBuyaHnl09fuA'

# Convert DataFrame to list of lists for Google Sheets
values = [df.columns.tolist()] + df.values.tolist()

# Specify the range where you want to update the data
range_name = 'RecentData!A1'  # This will start from A1 cell

# Update the Google Sheet
request = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=range_name,
    valueInputOption='RAW',
    body={'values': values}
)
response = request.execute()

print(f"{response.get('updatedCells')} cells updated.")

# Optional: Print the first few rows of the DataFrame
print(df.head())

168 cells updated.
   Month-Year  Total Users  Users/Active Users  Returning Users  New Users  \
18   11, 2024       1751.0              1440.0            209.0     1542.0   
15   10, 2024       2798.0              2785.0            245.0     2553.0   
13   09, 2024       2278.0              2257.0            244.0     2034.0   
11   08, 2024       2416.0              2393.0            234.0     2182.0   
9    07, 2024       2105.0              2087.0            231.0     1874.0   

    Pageviews  Hits/Events  Sessions  
18    12104.0      20745.0    2494.0  
15    21787.0      37547.0    4494.0  
13    22124.0      37073.0    3646.0  
11    21166.0      36546.0    3740.0  
9     22439.0      37983.0    3593.0  


In [3]:
# Get the spreadsheet
spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
sheet_id = spreadsheet['sheets'][0]['properties']['sheetId']

# Calculate the y-axis range
max_value = df[['Total Users', 'Users/Active Users', 'Returning Users', 'New Users']].max().max()
min_value = df[['Total Users', 'Users/Active Users', 'Returning Users', 'New Users']].min().min()
y_axis_max = max_value * 1.1  # Add 10% padding
y_axis_min = max(0, min_value * 0.9)  # Ensure it doesn't go below 0

# Define the chart with proper legend labels and improved y-axis
chart = {
    'spec': {
        'title': 'User Metrics Over Time for All Domains (GA4 Data)',
        'basicChart': {
            'chartType': 'LINE',
            'legendPosition': 'RIGHT_LEGEND',
            'headerCount': 1,
            'axis': [
                {'position': 'BOTTOM_AXIS', 'title': 'Month-Year'},
                {
                    'position': 'LEFT_AXIS',
                    'title': 'Count',
                    'viewWindowOptions': {
                        'viewWindowMin': y_axis_min,
                        'viewWindowMax': y_axis_max
                    }
                }
            ],
            'domains': [{
                'domain': {
                    'sourceRange': {
                        'sources': [{
                            'sheetId': sheet_id,
                            'startRowIndex': 0,
                            'endRowIndex': df.shape[0] + 1,
                            'startColumnIndex': 0,
                            'endColumnIndex': 1
                        }]
                    }
                },
                'reversed': True  # This will reverse the x-axis order
            }],
            'series': [
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 1,
                                'endColumnIndex': 2
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 0.4, 'green': 0.4, 'blue': 1.0},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 2,
                                'endColumnIndex': 3
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 1.0, 'green': 0.4, 'blue': 0.4},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 3,
                                'endColumnIndex': 4
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 1.0, 'green': 0.8, 'blue': 0.2},
                    'lineStyle': {'type': 'SOLID'}
                },
                {
                    'series': {
                        'sourceRange': {
                            'sources': [{
                                'sheetId': sheet_id,
                                'startRowIndex': 0,
                                'endRowIndex': df.shape[0] + 1,
                                'startColumnIndex': 4,
                                'endColumnIndex': 5
                            }]
                        }
                    },
                    'targetAxis': 'LEFT_AXIS',
                    'color': {'red': 0.2, 'green': 0.8, 'blue': 0.2},
                    'lineStyle': {'type': 'SOLID'}
                }
            ]
        }
    },
    'position': {
        'overlayPosition': {
            'anchorCell': {'sheetId': sheet_id, 'rowIndex': 0, 'columnIndex': 9},
            'widthPixels': 1200,
            'heightPixels': 600
        }
    }
}

# Add the chart to the sheet
chart_request = {
    'requests': [{
        'addChart': {
            'chart': chart
        }
    }]
}

response = service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body=chart_request
).execute()

print("New Chart added successfully.")

New Chart added successfully.


In [4]:
# Set up the query to fetch the traffic source data
traffic_source_request = RunReportRequest(
    property='properties/' + property_id,
    dimensions=[
        Dimension(name="year"),
        Dimension(name="month"),
        Dimension(name="sessionSource")
    ],
    metrics=[Metric(name="sessions")],
    order_bys=[
        OrderBy(dimension={'dimension_name': 'year'}, desc=True),
        OrderBy(dimension={'dimension_name': 'month'}, desc=True)
    ],
    date_ranges=[DateRange(start_date="2020-01-01", end_date="2024-09-30")]
)

# Send the request and get the response
traffic_source_response = client.run_report(traffic_source_request)

In [5]:
def format_traffic_source_report(response):
    data = {}
    for row in response.rows:
        year = int(row.dimension_values[0].value)
        month = int(row.dimension_values[1].value)
        source = row.dimension_values[2].value
        sessions = float(row.metric_values[0].value)
        
        key = f"{month:02}, {year}"
        if key not in data:
            data[key] = {"Organic Search": 0, "Direct": 0, "Referral": 0}
        
        if source.lower() == "google":
            data[key]["Organic Search"] += sessions
        elif source.lower() == "(direct)":
            data[key]["Direct"] += sessions
        elif source.lower() not in ["google", "(direct)"]:
            data[key]["Referral"] += sessions

    df = pd.DataFrame.from_dict(data, orient='index', columns=["Organic Search", "Direct", "Referral"])
    df.index.name = "Month-Year"
    df = df.reset_index()
    
    # Create a datetime column for sorting
    df['Year-Month'] = pd.to_datetime(df['Month-Year'].apply(lambda x: x.split(', ')[1] + '-' + x.split(', ')[0] + '-01'))
    
    # Sort by the datetime column in descending order
    df = df.sort_values(by=['Year-Month'], ascending=False)
    
    # Remove the sorting column and return the DataFrame
    return df[['Month-Year', 'Organic Search', 'Direct', 'Referral']]

# Apply the formatting to the traffic source data
traffic_source_df = format_traffic_source_report(traffic_source_response)

In [ ]:
# Convert DataFrame to list of lists for Google Sheets
traffic_source_values = [traffic_source_df.columns.tolist()] + traffic_source_df.values.tolist()

# Calculate the starting row for the new table
start_row = len(values) + 2  # Add 2 for an empty row between tables

# Specify the range for the new table
traffic_source_range = f'AllDomains_Data!A{start_row}'

# Update the Google Sheet with the traffic source data
traffic_source_request = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=traffic_source_range,
    valueInputOption='RAW',
    body={'values': traffic_source_values}
)
traffic_source_response = traffic_source_request.execute()

print(f"{traffic_source_response.get('updatedCells')} cells updated for traffic source data.")

# Optional: Print the first few rows of the traffic source DataFrame
print(traffic_source_df.head())

76 cells updated for traffic source data.
  Month-Year  Organic Search  Direct  Referral
0   09, 2024           977.0  1374.0    1380.0
1   08, 2024           805.0   948.0    1970.0
2   07, 2024           907.0  1204.0    1585.0
3   06, 2024           825.0  1116.0    1872.0
4   05, 2024           684.0  1128.0    2073.0


In [7]:
# Assuming traffic_source_df is already created and contains the data

# Get the current sheet properties
sheet_metadata = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
properties = sheet_metadata.get('sheets', [])[0].get('properties')
sheet_id = properties.get('sheetId')

# Calculate the position for the new chart
chart_start_row = start_row + len(traffic_source_values) + 2  # Add 2 for spacing

# Calculate the actual start row for the data (including header)
data_start_row = start_row - 1  # This will be the row with headers

requests = [{
    'addChart': {
        'chart': {
            'spec': {
                'title': 'Medium Traffic Source Sessions Over Time',
                'basicChart': {
                    'chartType': 'COLUMN',
                    'legendPosition': 'RIGHT_LEGEND',
                    'headerCount': 1,  # Explicitly tell the chart to use the first row as headers
                    'axis': [
                        {'position': 'BOTTOM_AXIS', 'title': 'Month-Year'},
                        {'position': 'LEFT_AXIS', 'title': 'Number of Sessions'}
                    ],
                    'domains': [{
                        'domain': {
                            'sourceRange': {
                                'sources': [{
                                    'sheetId': sheet_id,
                                    'startRowIndex': data_start_row,
                                    'endRowIndex': start_row + len(traffic_source_values) - 1,
                                    'startColumnIndex': 0,
                                    'endColumnIndex': 1
                                }]
                            }
                        }
                    }],
                    'series': [
                        {
                            'series': {
                                'sourceRange': {
                                    'sources': [{
                                        'sheetId': sheet_id,
                                        'startRowIndex': data_start_row,
                                        'endRowIndex': start_row + len(traffic_source_values) - 1,
                                        'startColumnIndex': 1,
                                        'endColumnIndex': 2
                                    }]
                                }
                            },
                            'targetAxis': 'LEFT_AXIS',
                            'color': {'red': 0.4, 'green': 0.6, 'blue': 0.9}  # Blue
                        },
                        {
                            'series': {
                                'sourceRange': {
                                    'sources': [{
                                        'sheetId': sheet_id,
                                        'startRowIndex': data_start_row,
                                        'endRowIndex': start_row + len(traffic_source_values) - 1,
                                        'startColumnIndex': 2,
                                        'endColumnIndex': 3
                                    }]
                                }
                            },
                            'targetAxis': 'LEFT_AXIS',
                            'color': {'red': 0.9, 'green': 0.3, 'blue': 0.3}  # Red
                        },
                        {
                            'series': {
                                'sourceRange': {
                                    'sources': [{
                                        'sheetId': sheet_id,
                                        'startRowIndex': data_start_row,
                                        'endRowIndex': start_row + len(traffic_source_values) - 1,
                                        'startColumnIndex': 3,
                                        'endColumnIndex': 4
                                    }]
                                }
                            },
                            'targetAxis': 'LEFT_AXIS',
                            'color': {'red': 1.0, 'green': 0.8, 'blue': 0.2}  # Yellow
                        }
                    ],
                    'stackedType': 'STACKED'
                }
            },
            'position': {
                'overlayPosition': {
                    'anchorCell': {
                        'sheetId': sheet_id,
                        'rowIndex': chart_start_row,
                        'columnIndex': 0
                    },
                    'widthPixels': 1000,
                    'heightPixels': 400
                }
            }
        }
    }
}]

# Execute the request to add the chart to Google Sheets
chart_response = service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body={'requests': requests}
).execute()

print("Stacked bar chart with proper legend added to Google Sheets successfully.")


Stacked bar chart with proper legend added to Google Sheets successfully.


In [8]:
# Set up the query to fetch top 10 pages by pageviews
top_pageviews_request = RunReportRequest(
    property='properties/' + property_id,
    dimensions=[Dimension(name="pagePath")],  # URL path of the pages
    metrics=[Metric(name="screenPageViews")],  # Metric: Pageviews
    order_bys=[OrderBy(metric={'metric_name': 'screenPageViews'}, desc=True)],  # Order by Pageviews (descending)
    date_ranges=[DateRange(start_date="2023-01-01", end_date="2024-09-30")],
    limit=10  # Limit to the top 10 results
)

# Send the request and get the response
top_pageviews_response = client.run_report(top_pageviews_request)

# Format the response into a DataFrame
def format_top_pages_report(response):
    row_headers = [row.dimension_values for row in response.rows]
    metric_values = [row.metric_values for row in response.rows]

    data = []
    
    for i in range(len(row_headers)):
        page_path = row_headers[i][0].value  # Page path (URL)
        pageviews = int(metric_values[i][0].value)  # Pageviews

        data.append([page_path, pageviews])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=["Page Path", "Pageviews"])

    return df

# Format the top 10 pages report
top_pages_df = format_top_pages_report(top_pageviews_response)
top_pages_df

,Page Path,Pageviews
0,/,57242
1,/glycan-search/,13216
2,/home/,6955
3,/protein-search/,6814
4,/index.php,2231
5,/quick-search/,2140
6,/glycan-search,2022
7,/super-search/,1988
8,/glycan/G49108TO,1823
9,/about/,1790


In [ ]:
# Convert DataFrame to list of lists for Google Sheets
top_pages_values = [top_pages_df.columns.tolist()] + top_pages_df.values.tolist()

# Calculate the starting row for the new table
# It should be after the traffic source data and chart
# Assuming chart takes about 25 rows of space
chart_space = 25
start_row_for_top_pages = start_row + len(traffic_source_values) + chart_space

# Specify the range for the new table
top_pages_range = f'AllDomains_Data!A{start_row_for_top_pages}'

# Add a title row before the table
title_range = f'AllDomains_Data!A{start_row_for_top_pages - 1}'
title_request = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=title_range,
    valueInputOption='RAW',
    body={'values': [['Top 10 Pages by Pageviews']]}
)
title_response = title_request.execute()

# Update the Google Sheet with the top pages data
top_pages_request = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=top_pages_range,
    valueInputOption='RAW',
    body={'values': top_pages_values}
)
top_pages_response = top_pages_request.execute()

# Format the title and table
requests = [
    {
        'updateCells': {
            'range': {
                'sheetId': sheet_id,
                'startRowIndex': start_row_for_top_pages - 1,
                'endRowIndex': start_row_for_top_pages,
                'startColumnIndex': 0,
                'endColumnIndex': 2
            },
            'rows': [{
                'values': [{
                    'userEnteredFormat': {
                        'textFormat': {'bold': True},
                        'horizontalAlignment': 'LEFT',
                        'backgroundColor': {'red': 0.9, 'green': 0.9, 'blue': 0.9}
                    }
                }]
            }],
            'fields': 'userEnteredFormat(textFormat,horizontalAlignment,backgroundColor)'
        }
    },
    {
        'updateCells': {
            'range': {
                'sheetId': sheet_id,
                'startRowIndex': start_row_for_top_pages,
                'endRowIndex': start_row_for_top_pages + 1,
                'startColumnIndex': 0,
                'endColumnIndex': 2
            },
            'rows': [{
                'values': [{
                    'userEnteredFormat': {
                        'textFormat': {'bold': True},
                        'backgroundColor': {'red': 0.95, 'green': 0.95, 'blue': 0.95}
                    }
                }]
            }],
            'fields': 'userEnteredFormat(textFormat,backgroundColor)'
        }
    }
]

# Execute the formatting requests
format_response = service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body={'requests': requests}
).execute()

print(f"Top pages data added to Google Sheets successfully.")


Top pages data added to Google Sheets successfully.


In [17]:
# Calculate the position for the new chart (next to the top pages table)
chart_start_row_top_pages = start_row_for_top_pages
# Calculate the actual start row for the data (including header)
data_start_row = start_row_for_top_pages - 1  # This will be the row with headers
requests = [{
    'addChart': {
        'chart': {
            'spec': {
                'title': 'Top 10 Pages by Pageviews',
                'basicChart': {
                    'chartType': 'BAR',  # Horizontal bar chart
                    'legendPosition': 'NO_LEGEND',  # Correct enum value for no legend
                    'axis': [
                        {
                            'position': 'BOTTOM_AXIS',
                            'title': 'Pageviews'
                        },
                        {
                            'position': 'LEFT_AXIS',
                            'title': 'Page Path'
                        }
                    ],
                    'domains': [{
                        'domain': {
                            'sourceRange': {
                                'sources': [{
                                    'sheetId': sheet_id,
                                    'startRowIndex': data_start_row,
                                    'endRowIndex': data_start_row + len(top_pages_values),
                                    'startColumnIndex': 0,  # Page Path column
                                    'endColumnIndex': 1
                                }]
                            }
                        }
                    }],
                    'series': [{
                        'series': {
                            'sourceRange': {
                                'sources': [{
                                    'sheetId': sheet_id,
                                    'startRowIndex': data_start_row,
                                    'endRowIndex': data_start_row + len(top_pages_values),
                                    'startColumnIndex': 1,  # Pageviews column
                                    'endColumnIndex': 2
                                }]
                            }
                        },
                        'targetAxis': 'BOTTOM_AXIS',
                        'color': {'red': 0.4, 'green': 0.6, 'blue': 0.9}  # Blue bars
                    }],
                    'headerCount': 1  # Use first row as headers
                }
            },
            'position': {
                'overlayPosition': {
                    'anchorCell': {
                        'sheetId': sheet_id,
                        'rowIndex': chart_start_row_top_pages,
                        'columnIndex': 3  # Position chart in column D
                    },
                    'widthPixels': 600,
                    'heightPixels': 400
                }
            }
        }
    }
}]

# Execute the request to add the chart
chart_response = service.spreadsheets().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body={'requests': requests}
).execute()

print("Top pages chart added successfully next to the table.")



Top pages chart added successfully next to the table.
